In [1]:
import numpy as np
from scipy.stats import unitary_group

#fixed!!
nDim = 2
Ns = 2
Nc = 2

Nx = 4
Nt = 4
lattice_volume = Nt * Nx

In [2]:
Pauli = []
Pauli.append(np.array([[0, 1], [1, 0]]))
Pauli.append(np.array([[0,-1j], [1j, 0]]))
Pauli.append(np.array([[1, 0], [0, -1]]))
Id = np.array([[1, 0], [0, 1]])

In [3]:
def RandomSU2matrix():
    tmp = np.random.rand(3)
    a = np.sqrt(sum(tmp*tmp))
    tmp /= a
    out = np.array([[0.j,0.j],[0.j,0.j]])
    for i in range(3):
        out += Pauli[i] * tmp[i]
    out *= np.sin(a) * 1j
    return Id * np.cos(a) + out

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m_q+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices, $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$, and $\hat\mu$ is a unit vector pointing in direction $\mu$.

Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity.

Calculating D, and inverting D is numerically slow, therefore we use the approximation where we can expand the inverse acting on a vector by a sum over applying D multiple times to this same vector. For this we need to Calculate D, which is sparse, and then compute the Matrix-Vector multiplication, which can be done for CSR Sparse matrices. Other way to do this (maybe not possible in the optical hardware?) is to directly apply D to the vector, and this is also implemented

In [4]:
psi = np.random.randn(lattice_volume, 2, 2)

In [5]:
gauge_links = np.zeros((lattice_volume, nDim, 2,2),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = RandomSU2matrix()

In [6]:
def LinearToLattice(array, Nt, Nx):
    return np.reshape(array, np.roll(np.append(np.array(array[0].shape), [Nx,Nt]),2))
def LatticeToLinear(array, lattice_volume):
    return np.reshape(array, np.roll(np.append(np.array(array[0,0].shape), lattice_volume),1))

In [7]:
def ConjugateTranspose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.conjugate(np.transpose(array, axes = axes))

In [8]:
def Transpose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.transpose(array, axes = axes)

In [9]:
def CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
    
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
    
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t = LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)
    gauge_links_shifted_x = LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5)
    
    
    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values_spinors = np.array([])
    values_colors = np.array([])
    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
        
        values_spinors = np.append(values_spinors, np.array([offdiagonal_spinor_t_minus, 
                                         offdiagonal_spinor_x_minus, np.sqrt(diagonal)*np.identity(Ns), 
                                                             offdiagonal_spinor_x_plus, offdiagonal_spinor_t_plus]))
        #We have to use sqrt of diagonal instead of diagonal, because we perform two matrix multiplication
        
#         values_colors = np.append(values_colors, np.array([gauge_links[(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt,0].T, 
#                                          gauge_links[(np.floor(i/4)*4)+((i%Nx)-1)%Nx,1].T, diagonal*np.identity(Nc), 
#                                                              gauge_links[i,1], 
#                                                            offdiagonal_spinor_t_plus]))
        values_colors = np.append(values_colors, np.array([ConjugateTranspose(gauge_links_shifted_t[i]), 
                                         ConjugateTranspose(gauge_links_shifted_x[i]), np.sqrt(diagonal)*np.identity(Nc), 
                                                             gauge_links[i,1], gauge_links[i,0]]))
    
        #I feel the transpose should be opposite, or idk
        
        #Maybe it's smart to add a value_diagonal, and separate all three, we could eliminate the diagonal term in col
        #and then we can choose the correct gauge link using the col_index. Or maybe we could move the gauge_link to right
        #The problem with the first approach, is that the positive u dont shift the gauge link. Therefore, maybe only
        #considere specific elements of col_index, namely the minus part and the diagonal part. 
        
        
    values_spinors = np.reshape(values_spinors, (lattice_volume*(2**nDim+1), Ns,Ns))
    values_colors = np.reshape(values_colors, (lattice_volume*(2**nDim+1), Nc,Nc))
    return row_index.astype(int), col_index.astype(int), values_spinors, values_colors

In [10]:
def SparseMatrixVectorMultiplication(rows, cols, values_spinors, values_colors, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(np.matmul(values_colors[row_elem + j],  input_vec[cols[row_elem + j]]),
                                        values_spinors[row_elem + j].T)
        out_vec[i] = output
    
    return out_vec

In [11]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)

In [12]:
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, values_colors, psi)

# psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, np.transpose(psi_new, axes = (0,2,1)))
# psi_new = np.transpose(psi_new, axes = (0,2,1))

In [13]:
def applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)

    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(ConjugateTranspose(gauge_links_shifted_t), psi_shifted_m_t)
    time_contribution_p = np.matmul(gauge_links[:,:,0,:,:], psi_shifted_p_t)
    space_contribution_m = np.matmul(ConjugateTranspose(gauge_links_shifted_x), psi_shifted_m_x)
    space_contribution_p = np.matmul(gauge_links[:,:,1,:,:], psi_shifted_p_x)

    # Estos matmuls son diferentes a los matmul
    time_contribution_m = np.matmul(time_contribution_m, Transpose(offdiagonal_spinor_t_minus))
    time_contribution_p = np.matmul(time_contribution_p, Transpose(offdiagonal_spinor_t_plus))
    space_contribution_m = np.matmul(space_contribution_m, Transpose(offdiagonal_spinor_x_minus))
    space_contribution_p = np.matmul(space_contribution_p, Transpose(offdiagonal_spinor_x_plus))
    

    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m + 
                                       time_contribution_p + space_contribution_m + space_contribution_p)
    
    return psi_new

In [14]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)

In [15]:
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, values_colors, psi)
# psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, np.transpose(psi_new, axes = (0,2,1)))
# psi_new = np.transpose(psi_new, axes = (0,2,1))

In [16]:
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)

In [17]:
np.sum(np.sqrt((LatticeToLinear(psi_new2,lattice_volume)-psi_new)**2))

(2.842170943040401e-14+0j)

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m_q+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices, $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$, and $\hat\mu$ is a unit vector pointing in direction $\mu$.

Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity.

# Test

In [18]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc))

In [19]:
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = np.identity(Nc)

In [20]:
for nt in range(2):
    for nx in range(2):
        a = 1
        p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
        #U = np.array([1,0,0])
        psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
        vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
        #vorfaktor[0,0] = 1
        #vorfaktor[0,1] = 1
        for t in range(Nt):
            for x in range(Nx):
                psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
        prefactor = (m + 2 - np.round(np.cos(a*p[1]))
                     - np.round(np.cos(a*p[0])))*np.identity(Ns) + 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
        manual_calculation = Transpose(np.matmul(prefactor, Transpose(psi)))
        psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
        print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

0j
0j
0j
0j


In [21]:
nt = 1 
nx = 1

a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
prefactor = (m + 2 - np.round(np.cos(a*p[1]))
             - np.round(np.cos(a*p[0])))*np.identity(Ns) + 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
manual_calculation = Transpose(np.matmul(prefactor, Transpose(psi)))
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

0j


In [22]:
psi_new2-manual_calculation

array([[[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


### Test # 2

In [23]:
U = np.diag(np.arange(3)+3)
U[0,2] = 12
U[2,0] = U[0,2]
U[0,1] = -111
U[1,0] = U[0,1]
U[1,2] = -34
U[2,1] = U[1,2]

In [24]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = U

In [25]:
for nt in range(5):
    for nx in range(5):
        a = 1
        p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
        #U = np.array([1,0,0])
        psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
        vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
        #vorfaktor[0,0] = 1
        #vorfaktor[0,1] = 1
        for t in range(Nt):
            for x in range(Nx):
                psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
        prefactor1 = (m + 2)*np.identity(Ns)  
        prefactor_cos = (- np.round(np.cos(a*p[1]))- np.round(np.cos(a*p[0])))*np.identity(Ns)
        prefactor_sin = 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
        prefactor2 = prefactor_cos + prefactor_sin
        manual_calculation1 = Transpose(np.matmul(prefactor1, Transpose(psi)))
        manual_calculation2 = np.matmul(U,Transpose(np.matmul(prefactor2, Transpose(psi))))
        manual_calculation = manual_calculation1 + manual_calculation2
        psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
        print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j
0j


In [26]:
nt = 2
nx = 3

a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
prefactor1 = (m + 2)*np.identity(Ns)  
prefactor_cos = (- np.round(np.cos(a*p[1]))- np.round(np.cos(a*p[0])))*np.identity(Ns)
prefactor_sin = 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
prefactor2 = prefactor_cos + prefactor_sin
manual_calculation1 = Transpose(np.matmul(prefactor1, Transpose(psi)))
manual_calculation2 = np.matmul(U,Transpose(np.matmul(prefactor2, Transpose(psi))))
manual_calculation = manual_calculation1 + manual_calculation2
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))


0j


In [27]:
(psi_new2-manual_calculation)

array([[[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


###### Wrong version

In [28]:

a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
prefactor1 = (m + 2)*np.identity(Ns)
prefactor_cos = (- np.round(np.cos(a*p[1])) - np.round(np.cos(a*p[0])))*np.identity(Ns)
prefactor_sin = 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
prefactor2 = prefactor_cos + prefactor_sin
manual_calculation = np.matmul(U,Transpose(np.matmul(prefactor2, Transpose(psi)))) 
+ Transpose(np.matmul(prefactor1, Transpose(psi))) # This enter here, was causing the problem!!!!! Careful
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))


0j


## Test # 3

In [63]:
U = unitary_group.rvs(3)

In [64]:
np.round(np.matmul(U, ConjugateTranspose(U)))

array([[ 1.+0.j, -0.-0.j, -0.-0.j],
       [-0.+0.j,  1.+0.j,  0.-0.j],
       [-0.+0.j,  0.+0.j,  1.+0.j]])

In [65]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = U

In [66]:
gamma_t_plus = np.identity(Ns) - Pauli[2] 
gamma_t_minus = np.identity(Ns) + Pauli[2] 
gamma_x_plus = np.identity(Ns) - Pauli[0] 
gamma_x_minus = np.identity(Ns) + Pauli[0] 

In [67]:
for nt in range(5):
    for nx in range(5):
        a = 1
        p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
        #U = np.array([1,0,0])
        psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
        vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
        #vorfaktor[0,0] = 1
        #vorfaktor[0,1] = 1
        for t in range(Nt):
            for x in range(Nx):
                psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
        prefactor_diagonal = (m + 2)*np.identity(Ns)  
        prefactor_plus = (-0.5*gamma_x_plus*np.round(np.exp(1j*a*p[1]))) -(0.5*gamma_t_plus*np.round(np.exp(1j*a*p[0])))
        prefactor_minus = (-0.5*gamma_x_minus*np.round(np.exp(-1j*a*p[1]))) - (0.5*gamma_t_minus*np.round(np.exp(-1j*a*p[0])))
        manual_calculation_diagonal = Transpose(np.matmul(prefactor_diagonal, Transpose(psi)))
        manual_calculation_plus = np.matmul(U,Transpose(np.matmul(prefactor_plus, Transpose(psi))))
        manual_calculation_minus = np.matmul(ConjugateTranspose(U),Transpose(np.matmul(prefactor_minus, Transpose(psi))))
        manual_calculation = manual_calculation_diagonal + manual_calculation_plus + manual_calculation_minus
        
        
        
        
        psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
        print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

(3.552713678800501e-15+0j)
0j
(2.5121479338940403e-15+0j)
0j
(3.552713678800501e-15+0j)
0j
0j
0j
0j
0j
(1.7763568394002505e-15+0j)
0j
0j
0j
(1.7763568394002505e-15+0j)
0j
0j
0j
0j
0j
(3.552713678800501e-15+0j)
0j
(2.5121479338940403e-15+0j)
0j
(3.552713678800501e-15+0j)


In [70]:
nt = 0
nx = 0
a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
prefactor_diagonal = (m + 2)*np.identity(Ns)  
prefactor_plus = (-0.5*gamma_x_plus*np.round(np.exp(1j*a*p[1]))) -(0.5*gamma_t_plus*np.round(np.exp(1j*a*p[0])))
prefactor_minus = (-0.5*gamma_x_minus*np.round(np.exp(-1j*a*p[1]))) - (0.5*gamma_t_minus*np.round(np.exp(-1j*a*p[0])))
manual_calculation_diagonal = Transpose(np.matmul(prefactor_diagonal, Transpose(psi)))
manual_calculation_plus = np.matmul(U,Transpose(np.matmul(prefactor_plus, Transpose(psi))))
manual_calculation_minus = np.matmul(ConjugateTranspose(U),Transpose(np.matmul(prefactor_minus, Transpose(psi))))
manual_calculation = manual_calculation_diagonal + manual_calculation_plus + manual_calculation_minus




psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

(3.552713678800501e-15+0j)


In [71]:
(psi_new2-manual_calculation)**2

array([[[[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]],

        [[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]],

        [[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]],

        [[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]]],


       [[[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]],

        [[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000000e+00+0.j, 7.88860905e-31+0.j],
         [0.00000000e+00+0.j, 0.00000000e+00+0.j]],

        [[0.00000000e+00+0.j, 0.00000000e+00+0.j],
         [0.00000

## Test # 4

In [36]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(3)

In [37]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
#gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)

In [38]:
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, values_colors, psi)
# psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, np.transpose(psi_new, axes = (0,2,1)))
# psi_new = np.transpose(psi_new, axes = (0,2,1))

In [39]:
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)

In [40]:
np.sum(np.sqrt((LatticeToLinear(psi_new2,lattice_volume)-psi_new)**2))

(3.5388358909926865e-14-3.469446951953614e-16j)

In [41]:
x

3

In [42]:
np.conjugate(x)

3

In [43]:
a = (1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1])))

In [44]:
a.shape

(2, 2)

In [45]:
nt = 2
nx = 1

a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
prefactor1 = (m + 2)*np.identity(Ns)
prefactor_cos = (- np.round(np.cos(a*p[1])) - np.round(np.cos(a*p[0])))*np.identity(Ns)
prefactor_sin = 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))
prefactor2 = prefactor_cos + prefactor_sin
manual_calculation = np.matmul(U,Transpose(np.matmul(prefactor2, Transpose(psi))))
+ Transpose(np.matmul(prefactor1, Transpose(psi)))
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

(5.019830309919167-9.243732780002638j)


In [46]:
psi_new2

array([[[[ 2.45533251-1.09323301j,  2.72501829-1.32810782j],
         [ 3.00840676-0.40212993j,  4.35241178-0.48116813j],
         [ 1.32617435+0.09696665j,  2.33163097+1.59389068j]],

        [[ 0.79308409+2.28168574j, -0.79041178+1.89474035j],
         [-1.59268915+3.72550123j,  0.19877027+3.30753351j],
         [-0.40046836+3.0790858j , -0.9533844 +1.93548928j]],

        [[-1.99035248-1.12732383j, -3.74983219-1.33477308j],
         [-1.86146737+0.03121913j, -2.53776903+1.25236865j],
         [-3.45960244-0.02040558j, -3.93141512+0.36721802j]],

        [[-0.39736677-3.47063151j,  0.47327159-3.87146645j],
         [ 1.48178217-3.68543659j,  2.48872516-3.23191004j],
         [-0.02075157-3.97862239j, -0.201684  -3.82928055j]]],


       [[[-1.82832084+0.81716652j, -1.43522262+1.36975723j],
         [-4.64611495-0.16454802j, -3.0430832 +1.83986529j],
         [-3.74701877+0.30609824j, -2.83893658+0.68034964j]],

        [[-1.29672802-2.0704714j , -1.48624642-3.5172068j ],
         [ 0

In [47]:
manual_calculation

array([[[[ 0.75039431+1.20045595j,  0.75039431+1.20045595j],
         [ 1.42356034+1.24252725j,  1.42356034+1.24252725j],
         [ 0.64875616-0.06731512j,  0.64875616-0.06731512j]],

        [[-1.20045595+0.75039431j, -1.20045595+0.75039431j],
         [-1.24252725+1.42356034j, -1.24252725+1.42356034j],
         [ 0.06731512+0.64875616j,  0.06731512+0.64875616j]],

        [[-0.75039431-1.20045595j, -0.75039431-1.20045595j],
         [-1.42356034-1.24252725j, -1.42356034-1.24252725j],
         [-0.64875616+0.06731512j, -0.64875616+0.06731512j]],

        [[ 1.20045595-0.75039431j,  1.20045595-0.75039431j],
         [ 1.24252725-1.42356034j,  1.24252725-1.42356034j],
         [-0.06731512-0.64875616j, -0.06731512-0.64875616j]]],


       [[[-0.75039431-1.20045595j, -0.75039431-1.20045595j],
         [-1.42356034-1.24252725j, -1.42356034-1.24252725j],
         [-0.64875616+0.06731512j, -0.64875616+0.06731512j]],

        [[ 1.20045595-0.75039431j,  1.20045595-0.75039431j],
         [ 1

In [48]:
prefactor

array([[3.+1.j, 0.+1.j],
       [0.+1.j, 3.-1.j]])

In [49]:
psi_new2

array([[[[ 2.45533251-1.09323301j,  2.72501829-1.32810782j],
         [ 3.00840676-0.40212993j,  4.35241178-0.48116813j],
         [ 1.32617435+0.09696665j,  2.33163097+1.59389068j]],

        [[ 0.79308409+2.28168574j, -0.79041178+1.89474035j],
         [-1.59268915+3.72550123j,  0.19877027+3.30753351j],
         [-0.40046836+3.0790858j , -0.9533844 +1.93548928j]],

        [[-1.99035248-1.12732383j, -3.74983219-1.33477308j],
         [-1.86146737+0.03121913j, -2.53776903+1.25236865j],
         [-3.45960244-0.02040558j, -3.93141512+0.36721802j]],

        [[-0.39736677-3.47063151j,  0.47327159-3.87146645j],
         [ 1.48178217-3.68543659j,  2.48872516-3.23191004j],
         [-0.02075157-3.97862239j, -0.201684  -3.82928055j]]],


       [[[-1.82832084+0.81716652j, -1.43522262+1.36975723j],
         [-4.64611495-0.16454802j, -3.0430832 +1.83986529j],
         [-3.74701877+0.30609824j, -2.83893658+0.68034964j]],

        [[-1.29672802-2.0704714j , -1.48624642-3.5172068j ],
         [ 0

In [50]:
manual_calculation

array([[[[ 0.75039431+1.20045595j,  0.75039431+1.20045595j],
         [ 1.42356034+1.24252725j,  1.42356034+1.24252725j],
         [ 0.64875616-0.06731512j,  0.64875616-0.06731512j]],

        [[-1.20045595+0.75039431j, -1.20045595+0.75039431j],
         [-1.24252725+1.42356034j, -1.24252725+1.42356034j],
         [ 0.06731512+0.64875616j,  0.06731512+0.64875616j]],

        [[-0.75039431-1.20045595j, -0.75039431-1.20045595j],
         [-1.42356034-1.24252725j, -1.42356034-1.24252725j],
         [-0.64875616+0.06731512j, -0.64875616+0.06731512j]],

        [[ 1.20045595-0.75039431j,  1.20045595-0.75039431j],
         [ 1.24252725-1.42356034j,  1.24252725-1.42356034j],
         [-0.06731512-0.64875616j, -0.06731512-0.64875616j]]],


       [[[-0.75039431-1.20045595j, -0.75039431-1.20045595j],
         [-1.42356034-1.24252725j, -1.42356034-1.24252725j],
         [-0.64875616+0.06731512j, -0.64875616+0.06731512j]],

        [[ 1.20045595-0.75039431j,  1.20045595-0.75039431j],
         [ 1

In [51]:
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, values_colors, psi)

IndexError: index 12 is out of bounds for axis 0 with size 4

In [ ]:
values_colors

In [ ]:
prefactor

In [ ]:
psi_new2

In [ ]:
U

In [ ]:
manual_calculation

In [ ]:
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')

In [ ]:
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)

In [ ]:
prefactor = (m + 2 - np.round(np.cos(a*p[1]))
             - np.round(np.cos(a*p[0])))*np.identity(Ns) + 1j*Pauli[2]*np.round(np.sin(a*p[0])) + 1j*Pauli[0]*np.round(np.sin(a*p[1]))

In [ ]:
manual_calculation = Transpose(np.matmul(prefactor, Transpose(psi)))

In [ ]:
np.sqrt(np.sum((psi_new2-manual_calculation)**2))

In [ ]:
np.cos(a*p[0])

In [ ]:
Pauli[2]

In [ ]:
1j*Pauli[2]*np.sin(a*p[0])

In [ ]:
Pauli[0]

In [ ]:
1j*Pauli[0]*np.sin(a*p[1])

In [ ]:
p

In [ ]:
p[1]*180/np.pi


In [ ]:
psi.shape

In [ ]:
prefactor.shape

In [ ]:
prefactor

In [ ]:
#psi

In [ ]:
print(psi[t,x,:,:])
print(manual_calculation[t,x,:,:])
print(psi_new2[t,x,:,:])

In [ ]:
np.cos(a*p[1])

In [ ]:
np.cos(a*p[0])

In [ ]:
Pauli[2]

In [ ]:
1j*Pauli[2]*np.sin(a*p[0])

In [ ]:
Pauli[0]

In [ ]:
1j*Pauli[0]*np.sin(a*p[1])

In [ ]:
p

In [ ]:
p[1]*180/np.pi


In [ ]:
psi.shape

In [ ]:
prefactor.shape

In [ ]:
prefactor

# Schmierpapier

Was wrong because the matrix multiplication was done in the wrong rows and cols, because after multiplying with U, it's already diagonal, and value of diagonal is multiplied twice if just implement the right matrix mult, because the diagonal term is considered twice, that's why take the square root

In [ ]:
# def SparseMatrixVectorMultiplication2(rows, cols, vals, input_vec):
#     out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
#     for i in range(input_vec[:,0,0].size):
#         row_elem = rows[i]
#         num_elements = rows[i+1] - rows[i]

#         output = np.zeros((vals.shape[-2], input_vec.shape[-1]),dtype = 'complex_')
#         for j in range(num_elements):
#             output = output + np.matmul(vals[row_elem + j],  input_vec[cols[row_elem + j]])
#         out_vec[i] = output
    
#     return out_vec

In [ ]:
# def applyD2(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
#     #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
#     if len(gauge_links) ==lattice_volume:
#         gauge_links = LinearToLattice(gauge_links, Nt,Nx)
#     if len(psi) ==lattice_volume:
#         psi = LinearToLattice(psi, Nt,Nx)

#     diagonal = (m+2)
#     #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
#     offdiagonal_spinor_x_plus = Id - Pauli[0]
#     offdiagonal_spinor_x_minus = Id + Pauli[0]
#     offdiagonal_spinor_t_plus = Id - Pauli[2]
#     offdiagonal_spinor_t_minus = Id + Pauli[2]

#     gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
#     gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)

#     #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
#     psi_shifted_m_t = np.roll(psi, 1, axis = 0)
#     psi_shifted_p_t = np.roll(psi, -1, axis = 0)
#     psi_shifted_m_x = np.roll(psi, 1, axis = 1)
#     psi_shifted_p_x = np.roll(psi, -1, axis = 1)


#     #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
#     time_contribution_m = np.matmul(np.transpose(gauge_links_shifted_t, axes = (0,1,3,2)), psi_shifted_m_t)
#     time_contribution_p = np.matmul(gauge_links[:,:,0,:,:], psi_shifted_p_t)
#     space_contribution_m = np.matmul(np.transpose(gauge_links_shifted_x, axes = (0,1,3,2)), psi_shifted_m_x)
#     space_contribution_p = np.matmul(gauge_links[:,:,1,:,:], psi_shifted_p_x)

#     # Estos matmuls son diferentes a los matmul
# #     time_contribution_m = np.matmul(time_contribution_m, Transpose(offdiagonal_spinor_t_minus))
# #     time_contribution_p = np.matmul(time_contribution_p, Transpose(offdiagonal_spinor_t_plus))
# #     space_contribution_m = np.matmul(space_contribution_m, Transpose(offdiagonal_spinor_x_minus))
# #     space_contribution_p = np.matmul(space_contribution_p, Transpose(offdiagonal_spinor_x_plus))


#     #Apply psi on the diagonal
#     self_contribution = diagonal*psi
#     #Add all
#     psi_new = self_contribution -0.5* (time_contribution_m + 
#                                        time_contribution_p + space_contribution_m + space_contribution_p)
    
#     return psi_new

In [ ]:
# Ns = 2
# Nc = 3
# m = 1
# psi = np.random.randn(lattice_volume,Nc,Ns)
# gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
# rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
# psi_new = SparseMatrixVectorMultiplication2(rows, cols,values_colors, psi)
# psi_new = SparseMatrixVectorMultiplication(rows, cols, values_spinors, np.transpose(psi_new, axes = (0,2,1)))
# psi_new = np.transpose(psi_new, axes = (0,2,1))

In [ ]:
# psi_new2 = applyD2(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)

In [ ]:
# psi_new

In [ ]:
# psi_new2

In [ ]:
# if len(phi) ==lattice_volume:
#     phi = np.reshape(phi, (Nx,Nt,phi.shape[-1]))
# if len(psi) ==lattice_volume:
#     psi = np.reshape(psi, (Nx,Nt,psi.shape[-1]))    
# diagonal = -m*np.ones(phi.shape) -g*phi
# offdiagonal1 = 1j*Pauli[0]
# offdiagonal2 = 1j*Pauli[2]

# a = np.array([-1,1])
# x_index = np.arange(Nx)
# t_index = np.arange(Nt)
# jx = (x_index[:,None]+np.array([-1,1]))%Nx
# jt = (t_index[:,None]+np.array([-1,1]))%Nt
# time_neighbors = psi[jt.T,:,:]
# space_neighbors = psi[:,jx.T,:]
# matriix = np.array([[[0,0],[0,0]] , [[1,1],[1,1]]])

# time_contribution = np.sum(time_neighbors, axis = 0)
# space_contribution = np.sum(space_neighbors, axis = 1)
# time_contribution = np.matmul(offdiagonal2,time_contribution[:,:,:,None]).squeeze()
# space_contribution = np.matmul(offdiagonal1,space_contribution[:,:,:,None]).squeeze()
# self_contribution = diagonal*psi
# psi_new = self_contribution + time_contribution + space_contribution

In [ ]:
# matriix = np.array([[[0,0],[0,0]] , [[1,1],[1,1]]])

# time_contribution = np.sum(time_neighbors, axis = 0)
# space_contribution = np.sum(space_neighbors, axis = 1)
# time_contribution = time_contribution[:,:,matriix]*offdiagonal2
# space_contribution = space_contribution[:,:,matriix]*offdiagonal1

In [ ]:
# out_vec = np.zeros(input_vec.shape)
# for i in range(input_vec[0].size):
#     row_elem = rows[i]
#     num_elements = rows[i+1] - rows[i]

#     output = 0.0
#     for j in range(num_elements):
#         output += np.matmul(vals[row_elem + j],  input_vec[cols[row_elem + j]])
#     out_vec[i] = output

# return out_vec

In [ ]:
# values = np.array([])
# for i in range(lattice_volume):
#     values = np.append(values, np.array([offdiagonal2, 
#                                          offdiagonal1, diagonal[i]*np.array([[1,0],[0,1]]), offdiagonal1, offdiagonal2]))

In [ ]:
# diagonal = -m*np.ones((lattice_volume,1)) -g*phi
# offdiagonal1 = 1j*Pauli[0]
# offdiagonal2 = 1j*Pauli[2]

In [ ]:
# values = np.array([])
# for i in range(lattice_volume):
#     values = np.append(values, np.array([offdiagonal2, 
#                                          offdiagonal1, diagonal[i]*np.array([[1,0],[0,1]]), offdiagonal1, offdiagonal2]))

In [ ]:
# col_group = np.array([-Nx,-1,0,1,Nx])
# col_index = np.array([])
# for i in range(lattice_volume):
#     col_index = np.append(col_index,
#                              np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
#                                        ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
#                                        (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))